In [4]:
import numpy as np
import pandas as pd
from sklearn import model_selection
# import liste des mots vides
from nltk.corpus import stopwords
# import fonction pour télécharger des ensemble de données
from nltk import download
import seaborn as sns
#Utiliser pour le traitement du langage naturel
import spacy
# Pour le rééquilibrage des classes
from imblearn.over_sampling import RandomOverSampler
import string
from sklearn.model_selection import train_test_split
import string
import tensorflow as tf

! pip install datasets
from datasets import Features, Value, ClassLabel, Dataset, DatasetDict
from transformers import AutoTokenizer
! pip install evaluate
import evaluate

from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [5]:

"""
Cette ligne charge le modèle de traitement du langage naturel pré-entraîné en français.
Ce modèle contient des informations telles que des vecteurs de mots et des règles de grammaire, qui sont utilisées pour analyser le texte en français.

"""
#utile pour télécharger les stopword
!python3 -m spacy download fr_core_news_sm
nlp=spacy.load('fr_core_news_sm')
download('stopwords')
stopWord=set(stopwords.words('french'))
print(stopWord)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 27.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
{'moi', 'sa', 'vos', 'à', 'ta', 'ce', 'serions', 'eu', 'eux', 'vous', 'étants', 'avons', 'fussions', 'des', 'serez', 'lui', 'étantes', 'ai', 'eusses', 'auras', 'fusse', 'étées', 'aient', 'du', 'avait', 'étée', 'êtes', 'j', 'm', 'étante', 'avaient', 'ton', 'eue', 'ma', 'un', 'soit', 'as', 'serait', 'pas', 'soyez', 'soient', 'pour', 'est', 'ou', 'seriez', 'c', 'aurai', 'ayants', 'la', 'aurait', 'toi', 'fusses', 'aura', 'qui', 'votre', 'étais', 'été', 'es', 's', 'serons', 'le', 'ayant', 'aies', 'avions', 'avais', 'qu', 'étions', 'ses', 'que', 'tes', 'aux', 'de', 'et', 'sommes', 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Importer les données

In [8]:
#Importer jeu d'entrainement
data_train = pd.read_csv('donnees/train.tsv',sep='\t')
data_train.info()
train = data_train[['headline','text','category']]

#Importer jeu de test
data_test= pd.read_csv('donnees/test.tsv',sep='\t')
test = data_test[['headline','text','category']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476 entries, 0 to 1475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1476 non-null   object
 1   headline  1476 non-null   object
 2   text      1476 non-null   object
 3   url       1476 non-null   object
dtypes: object(4)
memory usage: 46.2+ KB


# Préparer les données

## indexer les catégories

In [9]:
# Liste de catégorie  et ajout d'un identifiant numérique pour chaque catégorie
nom_categorie = sorted(train.category.unique().flatten())
print("Categorie:", nom_categorie)
print("\n")
label2id = {nom_categorie[i]:i for i in range(len(nom_categorie))}
id2label = {i:nom_categorie[i] for i in range(len(nom_categorie))}

print(label2id)
print("\n")
print(id2label)


Categorie: ['business', 'health', 'politics', 'sports', 'technology']


{'business': 0, 'health': 1, 'politics': 2, 'sports': 3, 'technology': 4}


{0: 'business', 1: 'health', 2: 'politics', 3: 'sports', 4: 'technology'}


## Préparer les données pour l'entrainer à un modèle

In [10]:
def preprocess(sentence) :
    list_w = nlp(sentence)
    list_w_clean = []
    res = []
    for token in list_w:
        if (token.text.lower() not in stopWord) and (token.text not in string.punctuation):
            list_w_clean.append(token)
    for token in list_w_clean:
        res.append(token.lemma_.lower())
    return " ".join(res)

In [16]:
data_df = pd.DataFrame()
# L'artcile est composé d'un headline et d'un texte
split_headline = train.headline

data_df['text'] = split_headline + ' ' + train.text
data_df['text']=data_df['text'].apply(preprocess)

# La classe cible est la catégorie sous forme d'identifiant numérique
data_df['category'] = train.category.map(label2id)

# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
province_features = Features({'text': Value('string'),
                              'category': ClassLabel(names=nom_categorie)})
data = Dataset.from_pandas(data_df, features=province_features)
# Découpage en train et test
data = data.train_test_split(test_size=0.2, shuffle=True, seed=12)


In [17]:
data['train'].features


{'text': Value(dtype='string', id=None),
 'category': ClassLabel(names=['business', 'health', 'politics', 'sports', 'technology'], id=None)}

In [18]:
data['train'][0]

{'text': 'guerre russie ukraine ville bombarder mère russie croire oleksandra quatre chien secours réfugier salle bain appartement kharkiv depuis début bombardement quand je entendre premier explosion je courir hors maison aller chercher chien leur enclore le extérieur gens paniquer abandonner leur voiture je tellement peur racont -t -elle jeune femme 25 an avoir parler régulièrement mère voir moscou cours conversation après avoir envoyer vidéo ville natal lourdement bombarder oleksandra parvenir convaincre mère danger que court vouloir effrayer parent je commencer dire directement civil enfant mourir dire lui se se inquiéter continuer dire celer ne arriver probablement accident le armée russe viser jamais civil ukrainien tuer propre peuple courant ukrainien de avoir famille le autre côté frontière russie certain comme oleksandra leur proche russe compréhension contraster conflit penser celer devoir histoire raconter média russe étroitement contrôler oleksandra dire mère faire répéter 

# Tokénisation des données

In [19]:
model_ckpt="camembert-base"
#Chargement du tokéniseur pré-entraîné correspondant au modèle utilisé
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)


In [21]:
# Tokénisation des 2 premières instances
preprocess_function(data['train'][:2])

{'input_ids': [[5, 775, 6358, 10, 6634, 21, 6218, 19323, 285, 14046, 81, 907, 6358, 10, 6634, 1757, 21, 2811, 8263, 21000, 700, 1877, 5213, 27474, 447, 1101, 2061, 21, 7951, 10530, 3089, 176, 479, 14046, 131, 206, 50, 3002, 246, 13420, 50, 5449, 1152, 269, 632, 1565, 1877, 97, 22, 216, 11379, 16, 3069, 497, 9018, 81, 13201, 97, 864, 50, 1217, 1020, 6720, 263, 67, 110, 67, 144, 393, 270, 618, 674, 190, 639, 1661, 907, 223, 3936, 10, 6512, 307, 4987, 182, 190, 3265, 646, 285, 12645, 23820, 14046, 81, 21, 2811, 8263, 21000, 4784, 5541, 907, 2873, 27, 1429, 2375, 21, 31531, 7514, 50, 1348, 248, 902, 4848, 851, 4647, 248, 111, 48, 48, 21, 13901, 1760, 248, 44, 1226, 45, 1871, 2154, 3962, 16, 6841, 3078, 18586, 283, 4848, 27476, 3866, 627, 1451, 1648, 27476, 8, 190, 401, 16, 238, 423, 4725, 6358, 10, 6634, 1000, 79, 21, 2811, 8263, 21000, 97, 1084, 3078, 4018, 8171, 81, 3814, 1278, 44, 1226, 2308, 738, 5160, 7428, 3078, 19240, 4785, 21, 2811, 8263, 21000, 248, 907, 85, 10733, 2567, 27, 3002,

In [22]:
# Tokenisation de la totalité des données : chaque unité est remplacée par un identifiant numérique
tokenized_data = data.map(preprocess_function, batched=True, batch_size=None)

Map:   0%|          | 0/1180 [00:00<?, ? examples/s]

Map:   0%|          | 0/296 [00:00<?, ? examples/s]

In [23]:
tokenized_data['train'][0]

{'text': 'guerre russie ukraine ville bombarder mère russie croire oleksandra quatre chien secours réfugier salle bain appartement kharkiv depuis début bombardement quand je entendre premier explosion je courir hors maison aller chercher chien leur enclore le extérieur gens paniquer abandonner leur voiture je tellement peur racont -t -elle jeune femme 25 an avoir parler régulièrement mère voir moscou cours conversation après avoir envoyer vidéo ville natal lourdement bombarder oleksandra parvenir convaincre mère danger que court vouloir effrayer parent je commencer dire directement civil enfant mourir dire lui se se inquiéter continuer dire celer ne arriver probablement accident le armée russe viser jamais civil ukrainien tuer propre peuple courant ukrainien de avoir famille le autre côté frontière russie certain comme oleksandra leur proche russe compréhension contraster conflit penser celer devoir histoire raconter média russe étroitement contrôler oleksandra dire mère faire répéter 

In [24]:
# Affichage des tokens. DistilBERT utilise l'algorithme WordPiece
tokens = tokenizer.convert_ids_to_tokens(tokenized_data['train'][0]['input_ids'])
print(tokenized_data['train'][0]['text'])
print(tokens)

guerre russie ukraine ville bombarder mère russie croire oleksandra quatre chien secours réfugier salle bain appartement kharkiv depuis début bombardement quand je entendre premier explosion je courir hors maison aller chercher chien leur enclore le extérieur gens paniquer abandonner leur voiture je tellement peur racont -t -elle jeune femme 25 an avoir parler régulièrement mère voir moscou cours conversation après avoir envoyer vidéo ville natal lourdement bombarder oleksandra parvenir convaincre mère danger que court vouloir effrayer parent je commencer dire directement civil enfant mourir dire lui se se inquiéter continuer dire celer ne arriver probablement accident le armée russe viser jamais civil ukrainien tuer propre peuple courant ukrainien de avoir famille le autre côté frontière russie certain comme oleksandra leur proche russe compréhension contraster conflit penser celer devoir histoire raconter média russe étroitement contrôler oleksandra dire mère faire répéter récit que 

In [25]:
# Taille du vocabulaire
tokenizer.vocab_size

32005

In [26]:
# Taille de contexte maximum
tokenizer.model_max_length

512

# Préparation de l'évaluation - Transformers

In [27]:
accuracy = evaluate.load("accuracy")

In [29]:
f1_metric = evaluate.load("f1")

In [30]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")
    return {"accuracy": acc['accuracy'], "f1-macro": f1["f1"]}

# Entrainement par affinage -- Fonctionne pas

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(
                    model_ckpt,
                    num_labels=len(nom_categorie),
                    id2label=id2label,
                    label2id=label2id,
                    ignore_mismatched_sizes=True
).to(device)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 64
training_args = TrainingArguments(
    output_dir=f"{model_ckpt}-finetuned-wine",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()